In [2]:
import os
import shutil
from PIL import Image # Para verificar dimensões da imagem para os bboxes
import matplotlib.pyplot as plt # Para visualização de bboxes
import matplotlib.patches as patches # Para desenhar retângulos
import cv2 # Usado na seção de verificação opcional, pode ser necessário para ler imagens

In [23]:
import os
import shutil

# --- Configurações de Caminho ---

# Define o caminho base onde todas as imagens serão CENTRALIZADAS.
# Crie este diretório se ele não existir.
# Exemplo: Se você quer que todas as imagens fiquem em 'AFO/all_images_here',
# defina ORIGINAL_DATA_BASE_PATH = 'AFO/all_images_here'
# VOU ASSUMIR QUE VOCÊ QUER JUNTAR TUDO NA RAIZ DO PROJETO OU EM UMA NOVA PASTA DEDICADA
# Ajuste conforme sua preferência, por exemplo, 'merged_images' ou 'all_dataset_images'
FINAL_MERGED_IMAGE_DIR = "all_dataset_images" # Nome da pasta para juntar todas as imagens

# Certifica-se de que a pasta de destino final existe
if not os.path.exists(FINAL_MERGED_IMAGE_DIR):
    os.makedirs(FINAL_MERGED_IMAGE_DIR)
    print(f"Pasta de destino '{FINAL_MERGED_IMAGE_DIR}' criada.")

# Diretórios de onde as imagens serão COLETADAS
image_dirs = [
    'PART_1/PART_1/images',
    'PART_2/PART_2/images',
    'PART_3/PART_3/images'
]

# Extensões de arquivo de imagem que serão consideradas
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')

# --- Processo de Junção de Imagens ---

print("\nIniciando a cópia das imagens...")
copied_image_count = 0

for source_dir in image_dirs:
    # Verifica se o diretório de origem existe antes de tentar listar
    if not os.path.exists(source_dir):
        print(f"Aviso: Diretório de origem '{source_dir}' não encontrado. Pulando...")
        continue

    print(f"Processando diretório: {source_dir}")
    for fname in os.listdir(source_dir):
        # Verifica se o arquivo é uma imagem pela extensão
        if fname.lower().endswith(IMAGE_EXTENSIONS):
            source_image_path = os.path.join(source_dir, fname)

            # Para evitar sobrescrever imagens com o mesmo nome vindo de pastas diferentes,
            # adicionamos um sufixo único ao nome do arquivo.
            # Uma forma simples é usar um contador ou adicionar o nome da pasta de origem.
            # Aqui, vou usar um contador global para manter os nomes curtos.
            
            base_name, extension = os.path.splitext(fname)
            destination_image_name = f"{base_name}{extension}"
            destination_image_path = os.path.join(FINAL_MERGED_IMAGE_DIR, destination_image_name)

            try:
                shutil.copy2(source_image_path, destination_image_path)
                # print(f"Copiado: {source_image_path} -> {destination_image_path}")
                copied_image_count += 1
            except Exception as e:
                print(f"ERRO ao copiar '{source_image_path}': {e}")

print(f"\nProcesso concluído. Total de {copied_image_count} imagens copiadas para '{FINAL_MERGED_IMAGE_DIR}'.")

Pasta de destino 'all_dataset_images' criada.

Iniciando a cópia das imagens...
Processando diretório: PART_1/PART_1/images
Processando diretório: PART_2/PART_2/images
Processando diretório: PART_3/PART_3/images

Processo concluído. Total de 3641 imagens copiadas para 'all_dataset_images'.


## 2. Funções Auxiliares

In [13]:
def load_class_names(filepath):
    """Carrega os nomes das classes do arquivo .names"""
    try:
        with open(filepath, 'r') as f:
            return [line.strip() for line in f.readlines()]
    except FileNotFoundError:
        print(f"Erro: Arquivo de nomes de classes não encontrado em {filepath}")
        return None

def get_target_indices(all_class_names, target_names):
    """Obtém os índices das classes de interesse"""
    if not all_class_names:
        return []
    target_indices = []
    for name in target_names:
        try:
            target_indices.append(all_class_names.index(name))
        except ValueError:
            print(f"Aviso: Classe de interesse '{name}' não encontrada na lista de classes do dataset.")
    return target_indices

def convert_yolo_to_absolute(x_center_norm, y_center_norm, width_norm, height_norm, img_width, img_height):
    """Converte coordenadas YOLO normalizadas para absolutas [xmin, ymin, xmax, ymax]."""
    w_abs = width_norm * img_width
    h_abs = height_norm * img_height
    x_min_abs = (x_center_norm * img_width) - (w_abs / 2)
    y_min_abs = (y_center_norm * img_height) - (h_abs / 2)
    x_max_abs = x_min_abs + w_abs
    y_max_abs = y_min_abs + h_abs
    return int(x_min_abs), int(y_min_abs), int(x_max_abs), int(y_max_abs)

## 3. Script Principal de Limpeza e Cópia

In [ ]:
def process_dataset():
    # Usar as variáveis globais definidas na Célula 4
    image_path_original = 'all_dataset_images'
    annotation_path_original = ANNOTATION_DIR_ORIGINAL
    class_names_filepath = CLASS_NAMES_FILE_ORIGINAL

    # O restante das definições de output_..._path já usa OUTPUT_ROOT_DIR, o que está correto
    output_image_path = os.path.join(OUTPUT_ROOT_DIR, OUTPUT_IMAGE_DIR_NAME)
    output_annotation_path = os.path.join(OUTPUT_ROOT_DIR, OUTPUT_ANNOTATION_DIR_NAME)

    # Verifica novamente se os caminhos de entrada são válidos
    if not os.path.isdir(image_path_original):
        print(f"ERRO: Diretório de imagens original '{image_path_original}' não encontrado. Verifique as configurações.")
        return
    if not os.path.isdir(annotation_path_original):
        print(f"ERRO: Diretório de anotações original '{annotation_path_original}' não encontrado. Verifique as configurações.")
        return
    if not os.path.isfile(class_names_filepath):
        print(f"ERRO: Arquivo de nomes de classes '{class_names_filepath}' não encontrado. Verifique as configurações.")
        return


    output_image_path = os.path.join(OUTPUT_ROOT, OUTPUT_IMAGE_DIR_NAME)
    output_annotation_path = os.path.join(OUTPUT_ROOT, OUTPUT_ANNOTATION_DIR_NAME)

    # Cria diretórios de saída se não existirem
    os.makedirs(output_image_path, exist_ok=True)
    os.makedirs(output_annotation_path, exist_ok=True)
    print(f"Diretório de saída para imagens limpas: {output_image_path}")
    print(f"Diretório de saída para anotações limpas: {output_annotation_path}")

    all_classes = load_class_names(class_names_filepath)
    if not all_classes:
        return

    target_indices = get_target_indices(all_classes, TARGET_CLASS_NAMES)
    if not target_indices:
        print("Nenhuma classe de interesse válida foi identificada. Saindo.")
        return

    print(f"\nClasses carregadas: {all_classes}")
    print(f"Índices das classes de interesse: {target_indices} ({[all_classes[i] for i in target_indices if i < len(all_classes)]})")

    processed_images_count = 0
    skipped_images_count = 0
    
    try:
        original_annotation_files = [f for f in os.listdir(annotation_path_original) if f.endswith('.txt')]
    except FileNotFoundError:
        print(f"ERRO: Não foi possível listar arquivos em '{annotation_path_original}'. O caminho está correto?")
        return
        
    print(f"Encontrados {len(original_annotation_files)} arquivos de anotação em '{annotation_path_original}'.")


    for ann_filename in original_annotation_files:
        base_filename = os.path.splitext(ann_filename)[0]
        
        # Tenta encontrar a imagem correspondente (comum ser .jpg, .png)
        # Adicione outras extensões se necessário (ex: .jpeg)
        possible_img_extensions = [".jpg", ".png", ".jpeg"]
        original_img_filepath = None
        img_extension = None

        for ext in possible_img_extensions:
            temp_img_path = os.path.join(image_path_original, base_filename + ext)
            if os.path.exists(temp_img_path):
                original_img_filepath = temp_img_path
                img_extension = ext
                break
        
        if not original_img_filepath:
            print(f"Aviso: Imagem correspondente para {ann_filename} não encontrada com extensões {possible_img_extensions}. Pulando.")
            continue
        
        current_ann_filepath = os.path.join(annotation_path_original, ann_filename)
        
        filtered_annotations = []
        try:
            with open(current_ann_filepath, 'r') as f_ann:
                lines = f_ann.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if not parts: # Linha vazia
                        continue
                    
                    try:
                        class_id = int(parts[0])
                        if class_id in target_indices:
                            # A linha já está no formato YOLO (classe_id x y w h)
                            # Apenas garantimos que tenha 5 partes (id + 4 coords)
                            if len(parts) == 5:
                                filtered_annotations.append(line.strip())
                            else:
                                print(f"Aviso: Linha em {ann_filename} com classe de interesse mas formato inesperado (num partes != 5): '{line.strip()}'")
                        
                    except (ValueError, IndexError):
                        print(f"Aviso: Linha mal formatada em {ann_filename}: '{line.strip()}'")
                        continue
        except Exception as e:
            print(f"Erro ao ler o arquivo de anotação {ann_filename}: {e}")
            continue

        if filtered_annotations:
            # Salva o novo arquivo de anotação
            new_ann_filepath = os.path.join(output_annotation_path, ann_filename)
            with open(new_ann_filepath, 'w') as f_new_ann:
                for ann_line in filtered_annotations:
                    f_new_ann.write(ann_line + '\n')
            
            # Copia a imagem correspondente
            new_img_filepath = os.path.join(output_image_path, base_filename + img_extension)
            try:
                shutil.copy2(original_img_filepath, new_img_filepath)
                processed_images_count += 1
            except Exception as e:
                print(f"Erro ao copiar a imagem {original_img_filepath} para {new_img_filepath}: {e}")
        else:
            skipped_images_count += 1
            # print(f"Imagem {base_filename} não continha objetos de interesse. Não será copiada.")

    print(f"\nProcessamento concluído.")
    print(f"Total de imagens processadas e salvas no novo dataset: {processed_images_count}")
    print(f"Total de imagens originais ignoradas (sem objetos de interesse ou erro na imagem): {skipped_images_count}")

# Executa a função principal de processamento
process_dataset()

Diretório de saída para imagens limpas: /datasets\images
Diretório de saída para anotações limpas: /datasets\labels

Classes carregadas: ['human', 'wind/sup-board', 'boat', 'bouy', 'sailboat', 'kayak']
Índices das classes de interesse: [0, 2, 4, 5] (['human', 'boat', 'sailboat', 'kayak'])
Encontrados 3641 arquivos de anotação em 'PART_1/PART_1\6categories'.
Aviso: Imagem correspondente para ev_1030.txt não encontrada com extensões ['.jpg', '.png', '.jpeg']. Pulando.
Aviso: Imagem correspondente para ev_1226.txt não encontrada com extensões ['.jpg', '.png', '.jpeg']. Pulando.
Aviso: Imagem correspondente para ev_363.txt não encontrada com extensões ['.jpg', '.png', '.jpeg']. Pulando.
Aviso: Imagem correspondente para ev_463.txt não encontrada com extensões ['.jpg', '.png', '.jpeg']. Pulando.
Aviso: Imagem correspondente para ev_546.txt não encontrada com extensões ['.jpg', '.png', '.jpeg']. Pulando.
Aviso: Imagem correspondente para ev_644.txt não encontrada com extensões ['.jpg', '.png

## 4. (Opcional) Verificação Visual do Bounding Box

Esta seção permite visualizar os bounding boxes de uma imagem específica **do dataset original** para confirmar se o formato das coordenadas (provavelmente YOLO: `classe_id x_centro_norm y_centro_norm largura_norm altura_norm`) está sendo interpretado corretamente.

**Instruções:**
1.  Certifique-se de que a célula de **Configurações** (Célula 4) e a de **Funções Auxiliares** (Célula 6) foram executadas.
2.  Altere a variável `IMAGE_TO_INSPECT_BASENAME` na célula de código abaixo para o nome base (sem extensão) de uma imagem que você deseja verificar.
3.  Execute a célula.

In [ ]:
# --- Configuração para Inspeção Visual ---
# Mude o nome do arquivo base da imagem que você quer inspecionar (sem .txt ou .jpg)
IMAGE_TO_INSPECT_BASENAME = "a_001" # EXEMPLO, MUDE PARA UM ARQUIVO SEU EXISTENTE

# --- Script de Inspeção ---
def inspect_bounding_boxes(image_basename):
    print(f"\nIniciando inspeção para: {image_basename}")
    
    # Recarregar nomes das classes caso esta célula seja executada independentemente (e as anteriores não)
    class_names_filepath_check = CLASS_NAMES_FILE_ORIGINAL # Usa a variável global
    all_classes_check = load_class_names(class_names_filepath_check)
    if not all_classes_check:
        print("Não foi possível carregar os nomes das classes para inspeção.")
        return

    annotation_filepath_check = os.path.join(ANNOTATION_DIR_ORIGINAL, image_basename + ".txt") # Usa a variável global
    
    # Tenta encontrar a imagem correspondente
    possible_img_extensions_check = [".jpg", ".png", ".jpeg"]
    img_filepath_check = None
    for ext_check in possible_img_extensions_check:
        temp_img_path_check = os.path.join(IMAGE_DIR_ORIGINAL, image_basename + ext_check) # Usa a variável global
        if os.path.exists(temp_img_path_check):
            img_filepath_check = temp_img_path_check
            break
            
    if not os.path.exists(annotation_filepath_check):
        print(f"Arquivo de anotação não encontrado para inspeção: {annotation_filepath_check}")
        return
    if not img_filepath_check:
        print(f"Arquivo de imagem não encontrado para inspeção (base: {image_basename})")
        return

    try:
        img = Image.open(img_filepath_check)
        img_width, img_height = img.size
        
        fig, ax = plt.subplots(1, figsize=(12, 9))
        ax.imshow(img)
        
        with open(annotation_filepath_check, 'r') as f:
            lines = f.readlines()
            if not lines:
                print(f"O arquivo de anotação {annotation_filepath_check} está vazio.")

            for line_idx, line in enumerate(lines):
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = int(parts[0])
                    x_center_norm = float(parts[1])
                    y_center_norm = float(parts[2])
                    width_norm = float(parts[3])
                    height_norm = float(parts[4])

                    abs_xmin, abs_ymin, abs_xmax, abs_ymax = convert_yolo_to_absolute(
                        x_center_norm, y_center_norm, width_norm, height_norm, img_width, img_height
                    )
                    
                    rect_width = abs_xmax - abs_xmin
                    rect_height = abs_ymax - abs_ymin
                    
                    # Define uma cor para o retângulo e texto
                    # Você pode criar um mapa de cores se quiser cores diferentes por classe
                    box_color = 'red'
                    if class_id < len(all_classes_check) and all_classes_check[class_id] in TARGET_CLASS_NAMES:
                        box_color = 'lime' # Verde para classes de interesse

                    rect = patches.Rectangle((abs_xmin, abs_ymin), rect_width, rect_height, linewidth=2, edgecolor=box_color, facecolor='none')
                    ax.add_patch(rect)
                    
                    class_name_to_display = all_classes_check[class_id] if class_id < len(all_classes_check) else f"ID_{class_id}"
                    
                    plt.text(abs_xmin, abs_ymin - 10, f'{class_name_to_display}', color=box_color, fontsize=10, bbox=dict(facecolor='white', alpha=0.5, pad=0))
                    print(f"  Obj {line_idx+1}: Classe: {class_name_to_display} (ID:{class_id}), YOLO norm: [{x_center_norm:.3f}, {y_center_norm:.3f}, {width_norm:.3f}, {height_norm:.3f}] -> Abs: [{abs_xmin}, {abs_ymin}, {abs_xmax}, {abs_ymax}]")
                else:
                    print(f"  Linha mal formatada ou com número incorreto de partes: {line.strip()}")

        plt.title(f"Inspeção de Bounding Boxes: {image_basename}")
        plt.axis('off') # Desliga os eixos para melhor visualização da imagem
        plt.show()

    except FileNotFoundError:
        print(f"Erro: Imagem ou arquivo de anotação não encontrado para {image_basename}")
    except Exception as e:
        print(f"Ocorreu um erro durante a inspeção de {image_basename}: {e}")

# Executa a inspeção para a imagem configurada
# Certifique-se de que as células de Configuração e Funções Auxiliares foram executadas antes
if IMAGE_TO_INSPECT_BASENAME: # Só executa se um nome de arquivo for fornecido
    inspect_bounding_boxes(IMAGE_TO_INSPECT_BASENAME)
else:
    print("Nenhum nome de arquivo fornecido em IMAGE_TO_INSPECT_BASENAME para inspeção.")


Iniciando inspeção para: a_001


NameError: name 'load_class_names' is not defined

## 5. Próximos Passos

Após executar este notebook:

1.  **Verifique a Saída:** Navegue até a pasta definida em `OUTPUT_ROOT`. Você deverá encontrar as subpastas `images` e `labels` (ou os nomes que você definiu) contendo apenas as imagens e anotações filtradas.
2.  **Use no PyTorch:** Agora você pode usar este dataset limpo para criar sua classe `Dataset` e `DataLoader` no PyTorch, conforme discutido anteriormente. As anotações estarão no formato YOLO (`classe_id x_centro y_centro largura altura` - todos normalizados), que é comum e pode ser facilmente lido e convertido para o formato que seu modelo PyTorch espera (geralmente tensores de bounding boxes `[xmin, ymin, xmax, ymax]` e labels).
3.  **Iteração:** Se necessário, ajuste as `TARGET_CLASS_NAMES` ou outras configurações e re-execute o notebook.